In [1]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import os
import pandas as pd
import configs
import torch
from sklearn.decomposition import PCA
from scipy.spatial.distance import pdist, squareform

from src.utils import find_ckpt_file, convert_to_tensor
import h5py
import random
from src.evals.eval_trees import EvalTrees


# Load Model

In [2]:
engram_dir = "/n/holylfs06/LABS/krajan_lab/Lab/cfang/icl-maze/"
wandb_project = "cntree"
corr = 0.25
env_name = f"cntree_layers7_bprob0.9_corr{corr}_state_dim10_envs300000_H800_explore"
dataset_storage_dir = f'{engram_dir}/{wandb_project}/{env_name}/datasets'
use_h5 = os.path.exists(os.path.join(dataset_storage_dir, 'test.h5'))
file_suffix = '.h5' if use_h5 else '.pkl'
test_dset_path = os.path.join(dataset_storage_dir, 'test' + file_suffix)

# Load Dataset and Create Environment

In [3]:
is_h5_file = test_dset_path.endswith('.h5')
if is_h5_file:
    eval_trajs = h5py.File(test_dset_path, 'r')
    traj_indices = list(eval_trajs.keys())
else:  # Pickle file
    with open(test_dset_path, 'rb') as f:
        eval_trajs = pickle.load(f)
n_eval_envs = len(eval_trajs)

# Run Model

In [10]:
query_every = 10
max_acc = []
for eval_traj in eval_trajs:
    seq_length = eval_traj['context_states'].shape[0]
    eval_idxs = np.concatenate([np.arange(0, seq_length, query_every), [seq_length-1]])
    opt_accuracy = np.ones(len(eval_idxs)) * (eval_traj['optimal_action'].squeeze().argmax()==0)
    first_reward = np.argwhere(eval_traj['context_rewards']>0)
    if first_reward.size > 0:
        first_reward = first_reward[0, 0]
        opt_accuracy[eval_idxs>first_reward] = 1
    max_acc.append(opt_accuracy.mean())

In [11]:
np.mean(max_acc)

np.float64(0.9534588477366256)